# Using Models and deployments
In this notebook, we use the WML client API to:
- List the models in a deployment space
- List the available deployments
- Score using the `AutoAI Churn Deployment` deployed model
- Promote and deploy the `AutoAI Price Estimate Model` model
- Score records using the `AutoAI Price Estimate Model` deployed model
- Create, save, deploy a model using sklearn, then score some data with it.
- List spaces
- List pipelines
- List experiments
- List runtimes


## Pre-requisites
It is assumes that you completed the previous part of the lab:
- Create and deploy an AutoAI churn model 
- Create an AutoAI price prediction model


## Watson Machine Learning APIs
See documentation: 
- Client API: https://wml-api-pyclient-dev-v4.mybluemix.net/
- REST API V2: https://cloud.ibm.com/apidocs/watson-data-api-cpd
- REST API V4: https://watson-ml-v4-api.mybluemix.net/

## Insert your WML service credentials below
Insert your username and password in the credential field

In [ ]:
import os, json, requests
# These are you credentials. You may want to remove them before sharing this notebook
credentials_url = os.getenv('RUNTIME_ENV_APSX_URL')
credentials_username = "username"
credentials_password = "password"

## Create a client connection

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

wml_credentials = {
    "instance_id": "openshift",
    "url": credentials_url,
    "username":credentials_username,
    "password": credentials_password,
    "version": "2.5.0"
}

client = WatsonMachineLearningAP1Client(wml_credentials)
print("Client version: " + client.version)

In [ ]:
user_id = os.getenv('USER_ID')
project_id = os.getenv('PROJECT_ID')
project_name = os.getenv('PROJECT_NAME')
accessToken = os.getenv('USER_ACCESS_TOKEN')
service_instance_url = client.service_instance.get_url()
mlInstanceGuid = client.service_instance.get_instance_id()


### Find the project deployment space

In [ ]:
# Get the guid for the space
spaces_details = client.spaces.get_details()

tag_value = "dsx-project." + project_id
mySpaces_details = next(item for item in spaces_details['resources']
                    if 'tags' in item['entity'] and item['entity']['tags'][0]['value'] == tag_value
                       )

space_uid = mySpaces_details['metadata']['guid']
space_name = mySpaces_details['entity']['name']
space_href = client.spaces.get_href(mySpaces_detail)
print(space_name + " guid: " + space_uid)
client.set.default_space(space_uid)

## List the models
To use the client API, we first need to sertup the defult space. This is done in the previous cell.

The method we use below only lists the information to the output. Other methods allow us to receive the output programmatically.

Note that the only model that is shown is the one that was promoted to the deployment space.

In [ ]:
client.repository.list_models()

## List deployments
The method we use below only lists the information to the output. Other methods allow us to receive the output programmatically.


In [ ]:
client.deployments.list()

## Score using the deployed model
In this case, we go through the available models and find the one we want using its name.<br/>
We can also get more details.

If there were multiple deployed models, we could find the model by name using the following code:
```
deployments_details = client.deployments.get_details()
deployed_details = next(item for item in deployments_details['resources']
                    if item['entity']["name"] == "AutoAI Churn Deployment")
deployed_uid = deployed_details['metadata']['guid']
```

Since we currently only have one deployed model, we can simply take the first one.

In [ ]:
deployments_details = client.deployments.get_details()
deployed_details = deployments_details['resources'][0]
                    
deployed_uid = deployed_detail['metadata']['guid']
deployed_name = deployed_detail['entity']['name']
print("Deployment name: " + deployed_name + ", uid: " + deployed_uid)

In [ ]:
# Get the related model details
model_details = client.repository.get_model_details(deployed_details['entity']['asset']['href'].split('/')[-1] )

# Input field names 
fieldnames = [x['name'] for x in model_details['entity']['schemas']['input'][0]['fields']] 

output = "["
for field in fildnames:
    output += '"' + field + '",'
output += ']'
output

In [ ]:
# Execute the model
scorring_payload = { client.deployments.ScoringMetaNames.INPUT_DATA:
                      [{
                         'fields': fieldnames,
                         'values': [[1,'F','S',1.0,38000.0,'N',24.393333,23.56,0.0,206.08,0.0,'CC','Budget','Intnl_discount',229.64,3.0],
                                    [6,'M','M',2.0,29616.0,'N',49.426667,29.78,0.0,45.5,0.0,  'CH','FreeLocal','Standard',75.29,2.0]
                                   ]
                        }]
                      }
predictions = client.deployments.score(deployed_uid, scoring_payload)

for prediction in predictions['predictions'] :
    for result in prediction['values'] :
        print('Prediction: {}, probability: [{}]'.format(result[0], result[1]))

## Deploy house pricing model
We have a saved model that we need to promote to the deployment space and then deploy. Then we can use it to score records.

We need to set the default project to see the model we want promote and deploy.

In [ ]:
client.set.default_project(project_id)

model_details = next(item for item in client.repository.get_model_details()['resources']
                    if item['entity']["name"] == "AutoAI Price Estimate Model")
model_uid = client.repository.get_model_uid(model_details)
print("Model name: " + model_details['entity']['name'] + ", uid: " + model_uid)

### Promote using the REST API

In [ ]:
url = service_instance_url + "/api/rest/catalogs/assets/" + model_uid + "/promote" 

headers = {
    'Authorization': "Bearer " + accessToken,
    'Content-Type': "application/json"
}

metadata = {
    "spaceId": space_uid,
    "projectId": project_id,
    "spaceName": space_name,
    "projectName": project_name,
    "assetType": "model",
    "mlInstanceGuid": mlInstanceGuid
}

payload = json.dumps(metadata, separators=(',', ':'))
promote_response = requests.post(url, 
                            data=payload,
                            headers=headers, 
                            verify=False
                        )

# print(promote_response.json());
promotedModelHref = promote_response.json()["href"]

promoted_model_response = requests.get(service_instance_url + promotedModelHref,
                            headers=headers, 
                            verify=False
                        )

# print(promoted_model_response.json());
promotedModelGuid = promoted_model_response.json()["metadata"]["asset_id"]
promotedModelHref = promoted_model_response.json()["href"]
promotedModelName = promoted_model_response.json()['metadata']['name']

print(promotedModelName + " guid: " + promotedModelGuid)

### Deploy using the REST API

In [ ]:
url = service_instance_url + "/v4/deployments"
deployment_name = promotedModelName + '_deployed'
 
metadata = {
    "name": deployment_name,
    "online": {},
    "space": { 
        "href": space_href
    },
    "asset": {
        "href": "/v4/models/" + promotedModelGuid + "?space_id=" + space_uid
    }
}
headers = {
    'Authorization': "Bearer " + accessToken,
    'Content-Type': "application/json"
}
payload = json.dumps(metadata, separators=(',', ':'))
 
deploy_response = requests.post(url, 
                            headers=headers, 
                            data=payload,
                            verify=False
                  )
deployment_details = deploy_response.json()
deployment_uid = client.deployments.get_uid(deployment_details)
# print(json.dumps(deployment_details, indent=4, sort_keys=True))
print("Deployment uid: " + deployment_uid)

## Score house pricing
We saw earlier that we can score records using the deployment information.

We can tie the deployment to the model information and use the model information to find out the input and output record formats and other information.

In [ ]:
# Input field names 
# Use the API to retrieve the field names used in the scoring call
fieldnames = [x['name'] for x in model_details['entity']['schemas']['input'][0]['fields']] 
', '.join(fieldnames)

In [ ]:
# Execute the model
client.set.default_space(space_uid)

scoring_payload = { client.deployments.ScoringMetaNames.INPUT_DATA:
                      [{
                         'fields': fildnames,
                         'values': [[0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.1,18.9],
                                    [0.22489,12.5,7.87,0,0.524,6.377,94.3,6.3467,5,311,15.2,392.52,20.45,15]
                                   ]
                        }]
                      }
predictions = client.deployments.score(deployment_uid, scoring_payload)
print("Predictions: {}".format([prediction['values'] for prediction in predictions['predictions']]))

### Remove the deployment

In [ ]:
client.deployments.delete(deployment_id)

## Create, save, promote and deploy a model

## Read training data

In [ ]:
import pandas as pd

url = 'https://github.com/jacquesroy/byte-size-data-science/raw/master/data/customer_churn.csv'
telco_df = pd.read_csv(url)
telco_df.head()

## Create a Churn model
Create a model and evaluiate it

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


In [ ]:
# Prepare pipeline to process categorical data and final processing pipeline
# We may want to add another preprocessing pipeline to handle numerical null values
categorical_features = ['Gender','Status','Car Owner','Paymethod','LocalBilltype','LongDistanceBilltype']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)])

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', DecisionTreeClassifier())])

In [ ]:
# Split into train and test
X = telco_df.drop('CHURN', axis=1)
y = telco_df['CHURN']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)

model = clf.fit(X_train, y_train)
res_predict = model.predict(X_test)
print("model score: %.3f" % clf.score(X_test, y_test))
print(classification_report(y_test, res_predict, target_names=["False", "True"]))

## Saving the model to WML

In [ ]:
# If we want to know the different aatributes that can be set when saving a model
client.repository.ModelMetaNames.git()

In [ ]:
# Checking the sklearn version we are using
import sklearn
sklearn.__version__

In [ ]:
metadata = {
  client.repository.ModelMetaNames.NAME: 'Telco Churn Prediction Model',
  client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.20',
  client.repository.ModelMetaNames.RUNTIME_UID: 'scikit-learn_0.20-py3'
}
# Name the columns
cols=["Gender", "Status", "Children", "Est Income", "Car Owner", "Age", "LongDistance",
      "International", "Local", "Dropped", "Paymethod", "LocalBilltype",
      "LongDistanceBilltype", "Usage", "RatePlan"]
      

saved_model = client.repository.store_model(model=model, meta_props=metadata, 
                                            # training_data=X_train, training_target=y_train, 
                                            feature_names=cols, label_column_names=["CHURN"] )
saved_model

In [ ]:
# See that it made it into WML
client.repository.list_models()

## Publish the model

In [ ]:
# Get the model UID
telco_model_uid = client.repository.get_model_uid(saved_model)

meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "Telco Churn Deployment",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

churn_deployment_details = client.deployments.created(telco_model_uid, meta_props)
churn_deployment_details

In [ ]:
# See that it was deployed
client.deployments.list()

## Accessing deployed model

In [ ]:
scoring_payload = {client.deployments.ScoringMetaNames.INPUT_DATA: 
                   [{'fields': ['ID','Gender','Status','Children','Est Income','Car Owner',
                              'Age','LongDistance','International','Local','Dropped',
                              'Paymethod','LocalBilltype','LongDistanceBilltype',
                              'Usage','RatePlan'],
                   'values': [[1,0,0,1.0,38000.0,'N',24.393333,23.56,0.0,206.08,0.0,'CC','Budget','Intnl_discount',229.64,3.0],
                              [6,1,'M',2.0,29616.0,'N',49.426667,29.78,0.0,45.5,0.0,  'CH','FreeLocal','Standard',75.29,2.0]
                             ]} ]}
deploy_uid = client.deployments.get_uid(churn_deployment_details)
predictions = client.deployments.score(deploy_uid, scorring_payload)
predictions

In [ ]:
for prediction in predictions['predictions'][0]['values'] :
    print("Prediction: {}, probability: {}".format(prediction[0],prediction[1]) )

## Cleanup
Remove the two models that we deployed in this notebook and the model we saved. This way, we reset the environment to where it was before we stated executing the notebook.

In [ ]:
# list the existing deployments to see what we currently have
client.deployments.list()

In [ ]:
# Retrieve the deployment details we want to remove
deployments_details = client.deployments.get_details()
model_deployed_details = next(item for item in deployments_details['resources']
                    if item['entity']["name"] == "Telco Churn Deployment")

client.deployments.delete(client.deployments.get_uid(model_deployed_details))

# See if the deployments were removed
client.deployments.list()

In [ ]:
# list the models currently in our WML service
client.repository.list_models()

In [ ]:
# We still have the saved_model variable that includes the model details.
client.repository.delete(saved_model['metadata']['guid'])
client.repository.list_models()

## Other Artifacts
We already saw models and deployments in details. We also just covered runtimes. There are other artifacts you should be aware of
- **runtimes**: List available runtimes
- **spaces**: WML includes one default space, we can define additional spaces so we can divide our artifacts more logically and add access control to them. 
- **function and libraries**: You can create functions that can be added to the environment and used in modeling
- **pipelines**: 
- **trainings**:
- **experiments**:



In [ ]:
# List runtimes
client.runtimes.list()

In [ ]:
# List spaces
client.repository.list_spaces()

In [ ]:
# List functions
client.repository.list_functions()

In [ ]:
# List pipelines
# The list is not empty since AutoAI used WML to train models
client.repository.list_pipelines()

In [ ]:
# List trainings
# This reflects the AutoAI model training
client.training.list()

In [ ]:
# List experiments
client.repository.list_experiments()

## WML Metadata
We can find out what metadata is available using the WML Client API. The following methods print out the metadata
- `client.repository.ModelMetaNames.show()`
- `client.repository.ExperimentMetaNames.show()`
- `client.repository.FunctionMetaNames.show()`
- `client.repository.PipelineMetaNames.show()`
- `client.repository.SpacesMetaNames.show()`


In [ ]:
# Example: pipeline metadata
client.repository.ModelMetaNames.show()

# Conclusion
WML facilitate the creating, sharing and running of models. Tools like AutoAI and Modeler Flows that use WML speed-up the development of machine learning models. Since WML is used through a freely available API, it can be used directly through programming interfaces such as a notebook or used to integrate it in other tools developped using languages like Java and Python.

There is also a REST API available that allows any application to score data with a deployed model.